# Multi Layer Perceptron for MNIST Dataset

### Perceptron 

- A perceptron is a linear classifer to separate data between two features
- Two possible output results:

    1) Positive = 1
    
    2) Negative = 0

In [20]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://sebastianraschka.com/images/blog/2015/singlelayer_neural_networks_files/perceptron_binary.png")

### Multi Layer Perceptron

- Feedfoward artificial neural network with one or more hidden layers
- Able to distinguish data that is not lineraly separable

In [18]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://www.safaribooksonline.com/library/view/getting-started-with/9781786468574/graphics/B05474_04_05.jpg")

### Hidden layers

- Called "hidden" layers because their values are not given in the input data
- The previous layer of the model provides a new representation of the input

In [19]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://i.stack.imgur.com/x9FAm.png")

In [1]:
from neon.util.argparser import NeonArgparser
from neon.backends import gen_backend
from neon.callbacks.callbacks import ProgressBarCallback
from neon.data import MNIST
from neon.data import ArrayIterator
from neon.data import MNIST
from neon.initializers import Gaussian
from neon.layers import Affine
from neon.transforms import Rectlin, Softmax
from neon.models import Model
from neon.layers import GeneralizedCost
from neon.transforms import CrossEntropyMulti
from neon.optimizers import GradientDescentMomentum
from neon.callbacks.callbacks import Callbacks
from neon.transforms import Misclassification

In [2]:
def load_data():
    """
    This method just downloads the pre-processed MNIST Dataset using Neon
    Built-in functions.
    """
    mnist = MNIST()
    (X_train, y_train), (X_test, y_test), nclass = mnist.load_data()
    return X_train, y_train, X_test, y_test

In [3]:
def get_Test_train(X_train,y_train,X_test,y_test,nclass):
    """
    This loads the Train and Test dataset into memory and returns an 
    Iterator for the model to use
    """
#     (num_examples, num_features) = (60000, 784).
    train_set = ArrayIterator(X_train, y_train, nclass=nclass)
#     (num_examples, num_features) = (10000, 784).
    test_set = ArrayIterator(X_test, y_test, nclass=nclass)
    return train_set, test_set

In [4]:
def initialize_weights():
    """
    This is a function to Initialize the weights using a Normal Gaussian Distribution 
    """
    init_norm = Gaussian(loc=0.0, scale=0.01)
    return init_norm

In [5]:
# Affine (i.e. fully-connected) layer 
# Normal Fully Connected Layer
def createAffine(n_hidden,activation_func,weight_init):
    """
    This method creates a AFFINE (Fully Connected Hidden Layer). 
    This is a normal linear layer used in Neural Networks.
    
    Params:
    @ n_hidden        : No of hidden units to use in the layer
    @ activation_func : The activation function to be used in the layer
    @ weight_init     : The weight Initializer to be used to initialize weights in this layer
    
    Returns:
    Affine Layer Object
    """
    return Affine(nout=n_hidden, init=weight_init, activation=activation_func)
    

In [6]:
# Model Architecture
def create_hidden_layers(layers,
                         no_of_layers,
                         hidden_unit_list,
                         activ_func_list,
                         weight_init_list):
    """
    This method creates the complete hidden layer architecture.
    
    Params:
    @ layers           : This is a list containing any previous layers or no layers 
    @ no_of_layers     : The no of hidden layers to create
    @ hidden_unit_list : A list of no of hidden units to use in each layer
    @ activ_func_list  : A list of Activation functions to be used in each layer
    @ weight_init_list : A list of weight initializers to be used in each layer
    
    Returns:
    A list of the newly created hidden layers
    """
    for index,num in enumerate(hidden_unit_list):
        layers.append(createAffine(n_hidden=num,
                                   weight_init=weight_init_list[index],
                                   activation_func=activ_func_list[index]))
    return layers


In [7]:
def init_model(layers):
    """
    This is a function that just initializes the Neural Network model by using the predefined layers.
    
    Params:
    @ layers : A list containing the different layer objects representing the Network
    
    Returns:
    An NN model Object
    """
    mlp = Model(layers=layers)
    return mlp

In [8]:
def init_cost(cost_func):
    """
    This is an initialization function to define the type of LOSS/COST function to use.
    
    Params:
    @ cost_func : the Type of Cost func to use
    
    Returns:
    A cost Function Object for the Network
    """
    cost = GeneralizedCost(costfunc=cost_func)
    return cost

In [9]:
def init_optimizer(learning_rate=0.1,momentum_coefficient=0.9):
    """
    Initilizes an Optimizer object 
    
    Params:
    @ learning_rate : The learning rate
    
    @ Momentum_cofficient: Regularization Parameter
    """
    optimizer = GradientDescentMomentum(learning_rate, momentum_coef=momentum_coefficient)
    return optimizer

In [10]:
# Neon provides an API for calling operations during the model fit (see Callbacks). 
# Here we set up the default callback, which is displaying a progress bar for each epoch.
def init_callback(model,evaluation_set):
    """
    Initializes Callback Function to check the Progress of the model during training
    
    Params:
    @ model : The NN model object
    @ evaluation_set: The Data to check against
    """
    callbacks = Callbacks(model, eval_set=evaluation_set)
#     callbacks.add_progress_bar_callback()
    callbacks.add_callback(ProgressBarCallback())
    return callbacks

In [11]:
def run_model(model,train_set,optimizer,epochs,cost,callbacks):
    """
    This function runs the model
    
    Params:
    @ model     : The model Object
    @ train_set : The training data set that contains both X,Y
    @ Optimizer : The optimizer to use for GD
    @ epochs    : The no of epochs to run for
    @ cost      : The cost Function to use
    @ callbacks : The callback function to check progress
    """
    model.fit(train_set, 
              optimizer=optimizer, 
              num_epochs= epochs,
              cost=cost,
              callbacks=callbacks)

In [12]:
def get_results(model,test_set):
    """
    This function retrieves the results
    """
    results = model.get_outputs(test_set)
    return results

In [13]:
def error_rate(model,test_set,metric):
    """
    This function checks the error rate of the model
    """
    error = model.eval(test_set, metric=metric)*100
    print('Misclassification error = %.1f%%' % error)

In [14]:
def main():
    
    gen_backend(backend='cpu', batch_size=128)
#     Step1: Load MNIST Data 
    X_train, y_train, X_test, y_test = load_data()
#     Step2: Create the Data Loader for test and train
    train_set, test_set = get_Test_train(X_train,y_train,X_test,y_test,nclass=10)
#     Step3: Create Layers for the model
    layers = []
    no_of_layers = 3
    hidden_unit_list = [30,50,10]
    activ_func_list = [Rectlin(),Rectlin(),Softmax()]
    weight_init_list = [initialize_weights() for i in range(len(hidden_unit_list))]
    layers = create_hidden_layers(layers,
                                  no_of_layers,
                                  hidden_unit_list,
                                  activ_func_list,
                                  weight_init_list)
    mlp = init_model(layers=layers)
    cost = init_cost(cost_func=CrossEntropyMulti())
   
    learning_rate = 0.1
    momentum_coef = 0.9
    
    optimizer = init_optimizer(learning_rate,momentum_coef)
    
    callback = init_callback(model=mlp,evaluation_set=test_set)
    
    run_model(model=mlp,
              train_set=train_set,
              optimizer=optimizer,
              epochs=10,
              cost=cost,
              callbacks=callback)
    
    results = get_results(model = mlp,test_set=test_set) 
    
    error_rate(model=mlp,test_set=test_set,metric=Misclassification())
    
    
    

In [15]:
main()

MemoryError: 

In [ ]:
# from neon.data import MNIST
# gen_backend(backend='cpu', batch_size=128)
# mnist = MNIST()
# train_set = mnist.train_iter
# test_set = mnist.valid_iter